In [2]:
# from ex_bnnstgp import BNN_model
import pandas as pd
import h5py
import numpy as np
import torch

/home/ellahe/.conda/envs/bnnstgp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Install package

In [1]:
pip install -i https://test.pypi.org/simple/ pkg-bnnstgp1

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pkg_bnnstgp1.pkg_bnnstgp import BNN_model

Start
using CPU


## Loading Data

In [4]:
Y = pd.read_csv("/nfs/turbo/jiankanggroup/ellahe/y1.csv").iloc[:,1].values
idx = np.invert(np.isnan(Y))
Y = Y[idx]

hf = h5py.File('/nfs/turbo/jiankanggroup/ellahe/image1.hdf5', 'r')
img1 = hf.get('img')['img1'][()][idx,:]
# img2 = hf.get('img')['img2'][()][idx,:]

h2 = h5py.File('/nfs/turbo/jiankanggroup/ellahe/coord1.hdf5', 'r')
coord1 = h2.get('coord')['coord1'][()]
# coord2 = h2.get('coord')['coord2'][()]

hf = h5py.File('/nfs/turbo/jiankanggroup/ellahe/image_fMRI2.hdf5', 'r')
img2 = hf.get('img')['img_fMRI'][()][idx,:]

h2 = h5py.File('/nfs/turbo/jiankanggroup/ellahe/coord_fMRI2.hdf5', 'r')
coord2 = h2.get('coord')['coord_fMRI'][()]

coord = [coord1, coord2]
img_data = [img1, img2]

In [5]:
BNN_neuroimg = BNN_model(coord=coord, imgs=img_data, cov=np.zeros((img_data[0].shape[0],1)),
                         Y=Y,rep=20,a=0.01,b=100, poly_degree=18, N_saves=70,
                         lamb=10,n_hid=128, n_hid2=16, lr=3e-3,path="/nfs/turbo/jiankanggroup/ellahe/multi_test_resize",nb_layer=2, n_epochs=151,
                        thred = 0.5, bg_image = "../data/neuroimaging/AAL_MNI_2mm.nii", batch_size=128,
                        nii_path = 'model_sig_nii/select_region_unit', n_img=len(img_data),
                        device='cuda' if torch.cuda.is_available() else 'cpu')

## Data preprocessing

In [5]:
BNN_neuroimg.load_data()

## Model training

Prespesified for traning 150 epochs

In [ ]:
R2 = BNN_neuroimg.train_model() 

Epoch 0, train time 82.6955 s, train MSE 1449.7301, train R2 0.000
  Epoch 0, test time 5.3566 s, test MSE 2.6474, test R2 0.020
Epoch 10, train time 97.3050 s, train MSE 980.0188, train R2 0.117
  Epoch 10, test time 5.5277 s, test MSE 2.5175, test R2 0.230
Epoch 20, train time 95.5218 s, train MSE 675.8970, train R2 0.237
  Epoch 20, test time 6.0616 s, test MSE 2.1076, test R2 0.291
Epoch 30, train time 94.0446 s, train MSE 466.1326, train R2 0.362
  Epoch 30, test time 5.9715 s, test MSE 1.9156, test R2 0.370
Epoch 40, train time 84.9024 s, train MSE 323.4803, train R2 0.425
  Epoch 40, test time 5.1118 s, test MSE 1.7593, test R2 0.465
Epoch 50, train time 88.9694 s, train MSE 226.6432, train R2 0.478
  Epoch 50, test time 5.6715 s, test MSE 1.6705, test R2 0.467


## Calculate Coverage Ratio

The three parts calculating coverage ratio, beta fdr control, and region selection, can only be accessed once the model is trained completely and stored in our pre-specified directory

In [ ]:
BNN_neuroimg.beta_post()

## Beta FDR control(used for next step's region selection)

Used for next step's region selection

In [6]:
BNN_neuroimg.beta_fdr_control()

Reptition: 0


KeyboardInterrupt: 

## Region Selection

In [7]:
BNN_neuroimg.output_selected()

IndexError: list index out of range